In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
df = pd.read_csv("creditcard.csv")

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [5]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [6]:
df['Class'].value_counts()

,count
Class,
0,284315
1,492


In [7]:
df.shape

(284807, 31)

In [8]:
# There are 284807 rows, 31 columns, no missing values. Class 0 has 284315 samples, Class 1 has 492 samples.

In [9]:
df0 = df[df['Class']==0]
df0.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
df0['Class'].value_counts()

,count
Class,
0,284315


In [11]:
y_train=df0['Class']
x_train= df0.drop('Class',axis=1)
y_test = df['Class']
x_test = df.drop('Class',axis=1)

In [12]:
scaler = MinMaxScaler()

In [13]:
cols = ['Time','Amount']
x_train[cols] = scaler.fit_transform(x_train[cols])
x_test[cols] = scaler.transform(x_test[cols])

In [14]:

input_dim = x_train.shape[1]

# Input layer
input_layer = Input(shape=(input_dim, ))

# Encoder
encoded = Dense(32, activation='relu')(input_layer)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(8, activation='relu')(encoded)

# Decoder
decoded = Dense(16, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(decoded)
output_layer = Dense(input_dim, activation='sigmoid')(decoded)


autoencoder = Model(inputs=input_layer, outputs=output_layer)


autoencoder.compile(optimizer=Adam(), loss='mse')


autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │           992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 30)             │           990 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,334 (13.02 KB)

 Trainable params: 3,334 (13.02 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:

history = autoencoder.fit(
    x_train, x_train,
    epochs=50,
    batch_size=32,
    shuffle=True,
)


Epoch 1/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - loss: 0.8319
Epoch 2/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - loss: 0.7553
Epoch 3/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - loss: 0.7629
Epoch 4/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - loss: 0.7441
Epoch 5/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - loss: 0.7337
Epoch 6/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - loss: 0.7332
Epoch 7/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - loss: 0.7291
Epoch 8/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - loss: 0.7329
Epoch 9/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - loss: 0.7337
Epoch 10/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - loss: 0.7336
Epoch 11/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - loss: 0.7298
Epoch 12/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - loss: 0.7363
Epoch 13/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - loss: 0.7266
Epoch 14/50
8885/8885 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - loss: 0.7498
Epoch 15/50
888

In [16]:
x_test_pred = autoencoder.predict(x_test)
mse = np.mean(np.power(x_test - x_test_pred, 2), axis=1)

8901/8901 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step


In [17]:
x_train_pred = autoencoder.predict(x_train)
mse_train = np.mean(np.power(x_train - x_train_pred, 2), axis=1)

threshold = np.max(mse_train)

8885/8885 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step


In [18]:
y_pred = []

for e in mse:
    if e > threshold:
        y_pred.append(1)  # anomaly (fraud)
    else:
        y_pred.append(0)  # normal


In [19]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[284315      0]
 [   492      0]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    284315
           1       0.00      0.00      0.00       492

    accuracy                           1.00    284807
   macro avg       0.50      0.50      0.50    284807
weighted avg       1.00      1.00      1.00    284807



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
